# Genetic optimisation of SVC classifier for the SDSS data

## Load data

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import warnings
import helpers
from helpers import DataSet
import matplotlib as mpl
import os
import random

# Common imports
import pandas as pd
import numpy as np
import seaborn as sns

# Imports for ML
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
sns.set_style('whitegrid')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "results"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# Helper functioins and structures
# Ignore useless warnings (see SciPy issue #5998)
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATA_PATH = "Skyserver_SQL2_27_2018 6_51_39 PM.csv"
RESULTS_FOLDER = "results"

# We load the data. Those that have nothing to do with the features of the objects are ignored.
sdss_data = pd.read_csv(DATA_PATH)

# We have a general look at the features
sdss_data.head(3)

print(sdss_data.columns.values)
sdss_data.drop(['objid', 'run', 'rerun', 'camcol', 'field',
                'specobjid', 'fiberid', 'mjd', 'plate'], axis=1, inplace=True)
sdss_data.head(1)

sdss_df_fe = sdss_data

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

X = sdss_data.drop(['class'], axis=1)
y = sdss_data['class']

class_names = ["GALAXY", "QSO", "STAR"]

std_scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_test = std_scaler.fit_transform(X_test.astype(np.float64))
X_train = std_scaler.fit_transform(X_train.astype(np.float64))

['objid' 'ra' 'dec' 'u' 'g' 'r' 'i' 'z' 'run' 'rerun' 'camcol' 'field'
 'specobjid' 'class' 'redshift' 'plate' 'mjd' 'fiberid']


## Optimize

In [2]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV

paramgrid = {
    "n_estimators": np.logspace(2, 3.5, num=10, base=10, dtype=int),
    "criterion": ['gini', 'entropy'],
    "min_samples_split": range(2, 10),
    "min_samples_leaf": range(1, 10),
    "max_features": ['auto', 'log2', None],
    "bootstrap": [True, False],
    "oob_score": [False, True],
    "random_state": [42]
}

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=ExtraTreesClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=5),
                                   verbose=3,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=15,
                                   n_jobs=8,
                                   error_score=0.0)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = cv.fit(X_train, y_train)

Types [1, 1, 1, 1, 1, 1, 1, 1] and maxint [9, 1, 7, 8, 2, 1, 1, 0] detected
[CV] n_estimators=215, criterion=gini, min_samples_split=6, min_samples_leaf=2, max_features=log2, bootstrap=False, oob_score=True, random_state=42 
[CV] n_estimators=2154, criterion=entropy, min_samples_split=5, min_samples_leaf=5, max_features=log2, bootstrap=False, oob_score=True, random_state=42 
[CV] n_estimators=146, criterion=entropy, min_samples_split=3, min_samples_leaf=3, max_features=None, bootstrap=False, oob_score=True, random_state=42 
[CV] n_estimators=464, criterion=gini, min_samples_split=3, min_samples_leaf=6, max_features=auto, bootstrap=True, oob_score=False, random_state=42 
[CV]  n_estimators=215, criterion=gini, min_samples_split=6, min_samples_leaf=2, max_features=log2, bootstrap=False, oob_score=True, random_state=42, score=0.0, total=   0.0s
[CV] n_estimators=215, criterion=gini, min_samples_split=6, min_samples_leaf=2, max_features=log2, bootstrap=False, oob_score=True, random_state=4

[CV]  n_estimators=464, criterion=gini, min_samples_split=3, min_samples_leaf=6, max_features=auto, bootstrap=True, oob_score=False, random_state=42, score=0.9646902065289806, total=   0.7s
[CV] n_estimators=464, criterion=gini, min_samples_split=3, min_samples_leaf=6, max_features=auto, bootstrap=True, oob_score=False, random_state=42 
[CV]  n_estimators=100, criterion=entropy, min_samples_split=2, min_samples_leaf=5, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9893262174783188, total=   0.4s
[CV] n_estimators=100, criterion=entropy, min_samples_split=2, min_samples_leaf=5, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=316, criterion=gini, min_samples_split=9, min_samples_leaf=1, max_features=log2, bootstrap=False, oob_score=False, random_state=42, score=0.9813457694870087, total=   0.8s
[CV] n_estimators=316, criterion=gini, min_samples_split=9, min_samples_leaf=1, max_features=log2, bootstrap=False, oob_sco

[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=4, max_features=None, bootstrap=True, oob_score=True, random_state=42 
[CV]  n_estimators=146, criterion=entropy, min_samples_split=5, min_samples_leaf=7, max_features=auto, bootstrap=True, oob_score=False, random_state=42, score=0.9586942038640907, total=   0.3s
[CV] n_estimators=146, criterion=entropy, min_samples_split=5, min_samples_leaf=7, max_features=auto, bootstrap=True, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=2, min_samples_leaf=4, max_features=None, bootstrap=True, oob_score=True, random_state=42, score=0.9853431045969354, total=   6.6s
[CV] n_estimators=2154, criterion=gini, min_samples_split=2, min_samples_leaf=4, max_features=None, bootstrap=True, oob_score=True, random_state=42 
[CV]  n_estimators=1000, criterion=gini, min_samples_split=8, min_samples_leaf=1, max_features=None, bootstrap=True, oob_score=True, random_state=42, score=0.987341772

[CV]  n_estimators=215, criterion=entropy, min_samples_split=6, min_samples_leaf=3, max_features=auto, bootstrap=True, oob_score=False, random_state=42, score=0.9773333333333334, total=   0.4s
[CV] n_estimators=215, criterion=entropy, min_samples_split=6, min_samples_leaf=3, max_features=auto, bootstrap=True, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=8, min_samples_leaf=1, max_features=log2, bootstrap=False, oob_score=False, random_state=42, score=0.9820119920053297, total=   5.6s
[CV] n_estimators=2154, criterion=gini, min_samples_split=8, min_samples_leaf=1, max_features=log2, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=215, criterion=entropy, min_samples_split=6, min_samples_leaf=3, max_features=auto, bootstrap=True, oob_score=False, random_state=42, score=0.9726484322881921, total=   0.4s
[CV] n_estimators=215, criterion=entropy, min_samples_split=6, min_samples_leaf=3, max_features=auto, bootstrap=True, o

[CV] n_estimators=100, criterion=entropy, min_samples_split=8, min_samples_leaf=5, max_features=auto, bootstrap=True, oob_score=False, random_state=42 
[CV]  n_estimators=100, criterion=entropy, min_samples_split=8, min_samples_leaf=5, max_features=auto, bootstrap=True, oob_score=False, random_state=42, score=0.9613075383589059, total=   0.2s
[CV] n_estimators=100, criterion=entropy, min_samples_split=8, min_samples_leaf=5, max_features=auto, bootstrap=True, oob_score=False, random_state=42 
[CV]  n_estimators=100, criterion=entropy, min_samples_split=8, min_samples_leaf=5, max_features=auto, bootstrap=True, oob_score=False, random_state=42, score=0.9673115410273516, total=   0.2s
[CV] n_estimators=3162, criterion=gini, min_samples_split=7, min_samples_leaf=7, max_features=auto, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=316, criterion=gini, min_samples_split=8, min_samples_leaf=6, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.

...

[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9926666666666667, total=   7.6s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9926666666666667, total=   7.7s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9926666666666667, total=   7.7s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_s

[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9880079946702198, total=   7.6s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9880079946702198, total=   7.7s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9913275517011341, total=  10.8s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_s

[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9886591060707138, total=  11.3s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9926666666666667, total=  11.4s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9926666666666667, total=  11.2s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_s

[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9880079946702198, total=  11.3s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9880079946702198, total=  11.2s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=auto, bootstrap=False, oob_score=False, random_state=42, score=0.9853333333333333, total=   7.8s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=auto, bootstrap=False, oob_s

[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9913275517011341, total=  11.2s
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9886591060707138, total=  11.4s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42, score=0.9886591060707138, total=  11.3s
[CV] n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=None, bootstrap=False, oob_score=False, random_state=42 
[CV]  n_estimators=2154, criterion=gini, min_samples_split=7, min_samples_leaf=1, max_features=auto, bootstrap=True, oob_s

In [3]:
best_indiv = LinearSVC(C = 5623.413251903491, 
                     penalty = 'l1',
                     loss = 'squared_hinge',
                     dual = False,
                     multi_class = 'ovr',
                     fit_intercept = True,
                     intercept_scaling = 0.03162277660168379,
                     random_state = 42,
                     tol = 1e-05,
                     max_iter = 316.22776601683796)